# &#x1F4D1; &nbsp;  Digit Recognition Models #2

## Links
[SciPy. Multi-dimensional image processing](https://docs.scipy.org/doc/scipy/reference/ndimage.html)

[Keras. Deep Learning library for Theano and TensorFlow](https://keras.io/)
 
[TensorFlow. Deep MNIST for Experts](https://www.tensorflow.org/get_started/mnist/pros)

[Tensorflow Deep MNIST Advanced Tutorial](http://docs.seldon.io/tensorflow-deep-mnist-example.html)

[Handwritten Digit Recognition using Convolutional Neural Networks in Python with Keras](http://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/)

## Other variants of this  project

 [Digit Recognition Models](https://olgabelitskaya.github.io/kaggle_digits.html)
 
 [Colaboratory Notebook](https://drive.google.com/open?id=1B1qh4ySXeJlWDMAXxAgHtS3jsyNdsmrn)

#### P5: Build a Digit Recognition Program

- https://olgabelitskaya.github.io/MLE_ND_P5_V0_S1.html

- https://olgabelitskaya.github.io/MLE_ND_P5_V0_S2.html

- https://olgabelitskaya.github.io/MLE_ND_P5_V0_S3.html

## Libraries

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from time import time

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pylab as plt
import matplotlib.cm as cm

%matplotlib inline

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model, neighbors, svm, ensemble
from sklearn import datasets, metrics 
from sklearn.model_selection import train_test_split

In [ ]:
from keras.utils import to_categorical
from keras.preprocessing import image as keras_image

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img

from keras import backend
from keras import losses
from keras.metrics import top_k_categorical_accuracy, categorical_accuracy
from keras.engine.topology import Layer
from keras.optimizers import Adam, Nadam
from keras.engine import InputLayer
from keras.models import Sequential, load_model, Model

from keras.layers import Input, BatchNormalization, Flatten, Dropout
from keras.layers import Dense, LSTM, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import UpSampling2D, Conv2DTranspose, DepthwiseConv2D
from keras.layers.core import RepeatVector, Permute
from keras.layers import Reshape, concatenate, merge

In [ ]:
from keras import __version__
print('keras version:', __version__)

## Datasets

In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")
df_train.shape

In [ ]:
df_train.ix[15:20,15:25]

In [ ]:
k = 0.48
images = ["%s%s" %("pixel",pixel_no) for pixel_no in range(0,784)]
train_images = np.array(df_train[images])
train_images = (train_images.astype('float32') / 255) ** k
#train_images = (train_images.astype('float32') / 255)
train_images.shape

In [ ]:
train_labels = df_train['label']
train_labels_cat = to_categorical(train_labels, num_classes=10)
train_labels_cat.shape

In [ ]:
test_images = np.array(df_test[images])
test_images = (test_images.astype('float32') / 255) ** k
test_images.shape

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(train_images, train_labels_cat, 
                 test_size=0.2, random_state=32)

In [ ]:
n = int(len(X_test)/2)
X_valid, y_valid = X_test[:n], y_test[:n]
X_test, y_test = X_test[n:], y_test[n:]
X_train.shape, X_test.shape, X_valid.shape, \
y_train.shape, y_test.shape, y_valid.shape

In [ ]:
y_train_num = np.array([np.argmax(x) for x in y_train])
y_test_num = np.array([np.argmax(x) for x in y_test])
y_valid_num = np.array([np.argmax(x) for x in y_valid])

## Examples

In [ ]:
fig, ax = plt.subplots(figsize=(14, 2), nrows=1, ncols=10, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(10):
    image = train_images[i].reshape(28,28)
    ax[i].imshow(image, cmap=plt.cm.Blues)

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.gcf()
ax[7].set_title('Examples of the 784-dimensional digits', fontsize=25);

## Models

***Model #1. Convolutional Neural Network. Keras***

In [ ]:
def top_3_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def cnn_model():
    model_input = Input(shape=(28, 28, 1))
    x = BatchNormalization()(model_input)
    
    x = Conv2D(28, (5, 5), padding='same')(x)
    x = LeakyReLU(alpha=0.02)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (5, 5))(x)
    x = LeakyReLU(alpha=0.02)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x) 

    x = GlobalMaxPooling2D()(x)
    
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.02)(x)
    x = Dropout(0.5)(x)
    
    y = Dense(10, activation='softmax')(x)
    
    model = Model(input=model_input, output=y)
    
    model.compile(loss='categorical_crossentropy', optimizer='nadam', 
                  metrics=[categorical_accuracy, top_3_categorical_accuracy])
    
    return model

In [ ]:
cnn_model = cnn_model()
print(cnn_model.summary())
cnn_checkpointer = ModelCheckpoint(filepath='weights.best.digits.cnn.hdf5', 
                                   verbose=2, save_best_only=True)
cnn_lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                     patience=5, verbose=2, factor=0.75)

In [ ]:
cnn_history = cnn_model.fit(X_train.reshape(-1, 28, 28, 1), y_train, 
                            validation_data=(X_valid.reshape(-1, 28, 28, 1), y_valid), 
                            epochs=75, batch_size=128, verbose=2, 
                            callbacks=[cnn_checkpointer, cnn_lr_reduction])

In [ ]:
cnn_model.load_weights('weights.best.digits.cnn.hdf5')
cnn_scores = cnn_model.evaluate(X_test.reshape(-1, 28, 28, 1), y_test, verbose=0)

print("CNN Scores: " , (cnn_scores))
print("CNN Error: %.2f%%" % (100 - cnn_scores[1]*100))

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(cnn_history.history['categorical_accuracy'][3:], '-o', label = 'train')
plt.plot(cnn_history.history['val_categorical_accuracy'][3:], '-o', label = 'test')
plt.legend()
plt.title('CNN Accuracy');

In [ ]:
steps, epochs = 1000, 20
data_generator = \
ImageDataGenerator(featurewise_std_normalization=True,
                   zoom_range=0.2, 
                   shear_range=0.2,
                   rotation_range=20,
                   height_shift_range=0.2,
                   width_shift_range=0.2)

dg_cnn_history = cnn_model.\
fit_generator(data_generator.flow(X_train.reshape(-1, 28, 28, 1), y_train, 
                                  batch_size=128),
              steps_per_epoch = steps, epochs = epochs,
              validation_data = (X_valid.reshape(-1, 28, 28, 1), y_valid), 
              callbacks=[cnn_checkpointer, cnn_lr_reduction], verbose=2)

In [ ]:
cnn_model.load_weights('weights.best.digits.cnn.hdf5')
cnn_scores = cnn_model.evaluate(X_test.reshape(-1, 28, 28, 1), y_test, verbose=0)

print("CNN Scores: " , (cnn_scores))
print("CNN Error: %.2f%%" % (100 - cnn_scores[1]*100))

***Model #2. Multi-layer Perceptron. Keras***

In [ ]:
def mlp_mc_model():
    model = Sequential()
    
    model.add(Dense(784, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.25))

    model.add(Dense(392, activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(196, activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='nadam', loss='categorical_crossentropy', 
                  metrics=[categorical_accuracy, top_3_categorical_accuracy])
    return model

In [ ]:
mlp_mc_model = mlp_mc_model()

In [ ]:
"""
fit_mlp = mlp_mc_model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                           epochs=3, batch_size=128, verbose=2);

plt.figure(figsize=(12,4))
plt.plot(fit_mlp.history['acc'], '-o', label = 'train')
plt.plot(fit_mlp.history['val_acc'], '-o', label = 'test')
plt.legend()
plt.title('MLP Accuracy');

mlp_scores = mlp_mc_model.evaluate(X_test, y_test)

print("\nMLP Scores: ", (mlp_scores))
print("MLP Error: %.2f%%" % (100 - mlp_scores[1] * 100))
print(mlp_mc_model.summary())
"""
""" """

***Model #3. Recurrent Neural Network. Keras***

In [ ]:
def rnn_mc_model():
    model = Sequential()

    model.add(LSTM(196, return_sequences=True, input_shape=(1, 784)))    
    model.add(LSTM(196, return_sequences=True))    
    model.add(LSTM(196))  
    
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='nadam', 
                  metrics=[categorical_accuracy, top_3_categorical_accuracy])    
    return model 

In [ ]:
rnn_mc_model = rnn_mc_model()

In [ ]:
"""
fit_rnn = rnn_mc_model.fit(X_train.reshape(X_train.shape[0], 1, X_train.shape[1]), y_train, 
                           epochs=3, batch_size=128, verbose=2, 
                           validation_data=(X_test.reshape(X_test.shape[0], 1, 
                                                           X_test.shape[1]), y_test))
plt.figure(figsize=(12,4))
plt.plot(fit_rnn.history['acc'], '-o', label = 'train')
plt.plot(fit_rnn.history['val_acc'], '-o', label = 'test')
plt.legend()
plt.title('RNN Accuracy');

rnn_scores = rnn_mc_model.evaluate(X_test.reshape(X_test.shape[0], 1, 
                                                  X_test.shape[1]), y_test)
print("\nRNN Scores: ", (rnn_scores))
print("RNN Error: %.2f%%" % (100 - rnn_scores[1] * 100))
print(rnn_mc_model.summary())
"""
""" """

***Model #4. MLPClassifier. Scikit-learn***

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(784,), max_iter=100, alpha=1e-4,
                     solver='lbfgs', verbose=1, tol=1e-6, random_state=1,
                     learning_rate_init=7e-4, batch_size=128)
clf.fit(X_train, y_train_num);

In [ ]:
print("MNIST. MLPClassifier. Train score: %f" % (clf.score(X_train, y_train_num)*100),'%')
print("MNIST. MLPClassifier. Test score: %f" % (clf.score(X_test, y_test_num)*100),'%')

## Predictions

In [ ]:
predict_labels = cnn_model.predict(test_images.reshape(28000,28,28,1))
predict_labels = predict_labels.argmax(axis=-1)

In [ ]:
submission = pd.DataFrame({"ImageId": range(1, len(predict_labels)+1), 
                           "Label": predict_labels})
print(submission[0:10])

submission.to_csv('kaggle_digits_cnn.csv', index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 2), nrows=1, ncols=10, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(10):
    image = test_images[i].reshape(28,28)
    ax[i].imshow(image, cmap=plt.cm.Blues)

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.gcf()
ax[4].set_title('Examples of the 784-dimensional digits. Test datapoints', fontsize=25);